In [1]:
# 1) load the dataset zip file in /content/sample_data dir
# 2) !unzip archive1.zip

!unzip '/content/sample_data/archive1.zip'

Archive:  /content/sample_data/archive1.zip
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_labels.csv  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/0.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/1.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/10.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/100.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/101.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/102.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/103.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/104.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/105.png  
  inflating: Doctor’s Handwritten Prescription BD dataset/Testin

In [ ]:
pip install pandas numpy scikit-learn tensorflow keras opencv-python

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


Num GPUs Available:  1


In [ ]:
# Load labels
data_folder = "/content/Doctors Handwritten Prescription BD dataset"
label_file = data_folder + '/Training/training_labels.csv'
labels_df = pd.read_csv(label_file)

# Print column names to verify
print(labels_df.columns)

Index(['IMAGE', 'MEDICINE_NAME', 'GENERIC_NAME'], dtype='object')


In [ ]:

import os
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


# Define the paths to the training images and labels
image_folder = data_folder + '/Training/training_words'
label_file = data_folder   + '/Training/training_labels.csv'

# Load labels
labels_df = pd.read_csv(label_file)

# Column names in the CSV file
filename_col = 'IMAGE'      # This is the actual column name
label_col = 'MEDICINE_NAME' # This is the actual column name

print("First few rows of the dataframe:")
print(labels_df.head())

# Initialize lists for images and labels
images = []
labels = []

# Load images and their corresponding labels
for index, row in labels_df.iterrows():
    img_path = os.path.join(image_folder, row[filename_col])
    img = load_img(img_path, target_size=(64, 64))  # Resize to a fixed size
    img = img_to_array(img)
    images.append(img)
    labels.append(row[label_col])

# Convert lists to numpy arrays
images = np.array(images, dtype="float") / 255.0  # Normalize pixel values
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded, num_classes=78)  

# Split the data
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

# Build the model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(78, activation='softmax'))  # 78 classes

# Compile the model
model.compile(optimizer=Adam(learning_rate =0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

model.save('prescription_classification_model.keras')


First few rows of the dataframe:
   IMAGE MEDICINE_NAME GENERIC_NAME
0  0.png         Aceta  Paracetamol
1  1.png         Aceta  Paracetamol
2  2.png         Aceta  Paracetamol
3  3.png         Aceta  Paracetamol
4  4.png         Aceta  Paracetamol


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.0040 - loss: 4.3816 - val_accuracy: 0.0128 - val_loss: 4.3594
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0205 - loss: 4.3459 - val_accuracy: 0.0449 - val_loss: 4.3038
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0658 - loss: 4.0871 - val_accuracy: 0.2644 - val_loss: 3.2625
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2696 - loss: 2.9866 - val_accuracy: 0.4728 - val_loss: 2.1203
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4844 - loss: 1.9803 - val_accuracy: 0.5657 - val_loss: 1.6483
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6339 - loss: 1.3542 - val_accuracy: 0.6394 - val_loss: 1.4338
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7280 - loss: 1.0027 - val_accuracy: 0.6843 - val_loss: 1.2880
Epoch 8/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7841 - loss: 0.7288 - val_accuracy: 0.7035 - val_loss

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

# Define the paths to the test images and labels
test_image_folder = data_folder + '/Testing/testing_words'
test_label_file = data_folder + '/Testing/testing_labels.csv'

test_labels_df = pd.read_csv(test_label_file)

# Column names in the CSV file
filename_col = 'IMAGE'
label_col = 'MEDICINE_NAME'

# Debug: print the first few rows to understand the structure
print("First few rows of the test dataframe:")
print(test_labels_df.head())

# Initialize lists for test images and labels
test_images = []
test_labels = []

# Load test images and their corresponding labels
for index, row in test_labels_df.iterrows():
    img_path = os.path.join(test_image_folder, row[filename_col])
    img = load_img(img_path, target_size=(64, 64))  # Resize to a fixed size
    img = img_to_array(img)
    test_images.append(img)
    test_labels.append(row[label_col])

# Convert lists to numpy arrays
test_images = np.array(test_images, dtype="float") / 255.0  # Normalize pixel values
test_labels = np.array(test_labels)

# Encode test labels
label_encoder = LabelEncoder()
test_labels_encoded = label_encoder.fit_transform(test_labels)
test_labels_categorical = tf.keras.utils.to_categorical(test_labels_encoded, num_classes=78)  # 78 classes

# Load the trained model that was saved earlier
model = load_model('prescription_classification_model.keras')

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_images, test_labels_categorical)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Predict the classes
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels_categorical, axis=1)

# Generate classification report
print(classification_report(true_classes, predicted_classes, target_names=label_encoder.classes_))

# Generate confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)


First few rows of the test dataframe:
   IMAGE MEDICINE_NAME GENERIC_NAME
0  0.png         Aceta  Paracetamol
1  1.png         Aceta  Paracetamol
2  2.png         Aceta  Paracetamol
3  3.png         Aceta  Paracetamol
4  4.png         Aceta  Paracetamol
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.5818 - loss: 2.6732
Test Accuracy: 52.05%
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
              precision    recall  f1-score   support

         Ace       0.58      0.70      0.64        10
       Aceta       0.28      0.80      0.41        10
     Alatrol       0.42      0.50      0.45        10
      Amodis       0.48      1.00      0.65        10
     Atrizin       0.43      0.30      0.35        10
      Axodin       0.70      0.70      0.70        10
          Az       0.75      0.90      0.82        10
  Azithrocin       1.00      0.60      0.75        10
       Azyth       0.75      0.90      0.82        10
      Bacaid       0.60      0.60      0.60        10
    Backtone   

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Fit the generator to the training data
datagen.fit(X_train)

# Train the model with augmented data
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test), epochs=20)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


78/78 ━━━━━━━━━━━━━━━━━━━━ 11s 102ms/step - accuracy: 0.1489 - loss: 4.4232 - val_accuracy: 0.5369 - val_loss: 1.9365
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 13s 39ms/step - accuracy: 0.2528 - loss: 3.1859 - val_accuracy: 0.6635 - val_loss: 1.4719
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.2977 - loss: 2.9155 - val_accuracy: 0.6538 - val_loss: 1.4036
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.2742 - loss: 2.8628 - val_accuracy: 0.6042 - val_loss: 1.5620
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - accuracy: 0.3279 - loss: 2.6281 - val_accuracy: 0.6859 - val_loss: 1.3393
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.3636 - loss: 2.4421 - val_accuracy: 0.6747 - val_loss: 1.2599
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.3967 - loss: 2.3486 - val_accuracy: 0.6779 - val_loss: 1.2837
Epoch 8/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.3996 - loss: 2.3009 - val_accuracy: 0.7051 - val_loss: